In [89]:
import pandas as pd
import plotly.express as px
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import re
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import datetime as dt
import json
from pandas.io.json import json_normalize

In [23]:
injuries_url = 'https://www.fantasyfootballscout.co.uk/fantasy-football-injuries/'

# Uses pandas built-in read_html function to grab a list of all html tables from injuries_url
# You can print injury_tables to check the output here if you like
# In this case there is only one table on the page, but sometimes you will have a few
injury_tables = pd.read_html(injuries_url, encoding='utf-8')

In [24]:
# Select the first table from the injury_tables list
# Note that in Python data structures the first item is always at index 0, not index 1 (which is the second item)
injuries = injury_tables[0]

# Split the string inside the 'Name' column on the open bracket character
# Will return a list, e.g. [Cech, Petr)] (uncomment the next line to check if you like)
# injuries['split_checker'] = injuries['Name'].str.split('(')
# Then use str.get() to grab the first item in the list and save it to a new column, 'last_name'
injuries['last_name'] = injuries['Name'].str.split('(').str.get(0)
injuries['first_name'] = injuries['Name'].str.split('(').str.get(1).str.strip(')')
injuries['full_name'] = injuries['first_name'] + ' ' + injuries['last_name']

# Remove accented characters (see https://stackoverflow.com/questions/37926248/how-to-remove-accents-from-values-in-columns)
injuries['last_name'] = injuries['last_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
 
# Set full name to equal last name when there is an error
# This will take care of players that only use one name (e.g. Kenedy)
# However, if we were trying to match the names to another source players like Lowe (Chris Lowe) and Alli (Dele Alli) could cause problems
injuries['full_name'] = injuries['full_name'].fillna(injuries['last_name'])

injuries.columns = injuries.columns.str.lower().str.replace(' ', '_')

# Convert 'last_updated' column to datetime
# We can specify the format of the date if necessary, but often pandas will convert it automatically
# In this case, we don't want it to get confused between US and UK formats
injuries['last_updated'] = pd.to_datetime(injuries['last_updated'], format='%d/%m/%Y')

injuries.head(10)

,name,club,status,return_date,latest_news,last_updated,last_name,first_name,full_name
0,Chambers (Calum),ARS,Injured,01/07/2020,Knee injury Limped off with a serious knee inj...,2020-01-09,Chambers,Calum,Calum Chambers
1,Elneny (Mohamed),ARS,On Loan,01/07/2020,Joined Besiktas on loan until the end of the s...,2020-02-01,Elneny,Mohamed,Mohamed Elneny
2,Jenkinson (Carl),ARS,Unavailable,Unknown,Joined Nottingham Forest on a permanent deal o...,2019-08-07,Jenkinson,Carl,Carl Jenkinson
3,John-Jules (Tyreece),ARS,On Loan,01/07/2020,Joined Lincoln City on loan until the end of t...,2020-01-16,John-Jules,Tyreece,Tyreece John-Jules
4,Kolasinac (Sead),ARS,Doubt 75%,04/04/2020,Shoulder injury Substituted after suffering a ...,2020-03-13,Kolasinac,Sead,Sead Kolasinac
5,Koscielny (Laurent),ARS,Unavailable,Unknown,Joined Bordeaux on a permanent deal on 6/8.,2019-08-08,Koscielny,Laurent,Laurent Koscielny
6,Mavropanos (Konstantinos),ARS,On Loan,01/07/2020,Joined Nuremberg on loan until the end of the ...,2020-01-13,Mavropanos,Konstantinos,Konstantinos Mavropanos
7,Mkhitaryan (Henrikh),ARS,On Loan,01/07/2020,Joined Roma on loan until the end of the seaso...,2019-09-02,Mkhitaryan,Henrikh,Henrikh Mkhitaryan
8,Monreal (Nacho),ARS,Unavailable,Unknown,Joined Real Sociedad on a permanent deal on 31...,2019-08-31,Monreal,Nacho,Nacho Monreal
9,Smith-Rowe (Emile),ARS,On Loan,01/07/2020,Joined Huddersfield Town on loan until the end...,2020-01-10,Smith-Rowe,Emile,Emile Smith-Rowe


In [31]:
today = dt.date.today()
one_week_ago = today - dt.timedelta(days=7)
recent_injuries = injuries[injuries['last_updated'] >= one_week_ago]

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.



In [32]:
xg_url = 'https://understat.com/league/EPL'
xg_data = requests.get(xg_url)
xg_html = xg_data.content
soup = BeautifulSoup(xg_html, 'lxml')
# It's good practice to try and put any extra code inside a new cell, so you don't have to make a request to the page more than once
# If you keep running this cell it will make a new request to the site every time

In [37]:
#soup.prettify()
soup.title

<title>EPL xG Table and Scorers for the 2019/2020 season | Understat.com</title>

In [50]:
# Set up the Selenium driver (in this case I am using the Chrome browser)
options = webdriver.ChromeOptions()
 
# 'headless' means that it will run without opening a browser
# If you don't set this option, Selenium will open up a new browser window (try it out if you like)
options.add_argument('headless')
 
# Tell the Selenium driver to use the options we just specified
driver = webdriver.Chrome(executable_path='C:/Selenium/chromedriver.exe', options=options)
 
# Tell the driver to navigate to the page url
driver.get(xg_url)
 
# Grab the html code from the webpage
soup = BeautifulSoup(driver.page_source, 'lxml')

In [56]:
# Get the table headers using 3 chained find operations
# 1. Find the div containing the table (div class = chemp jTable)
# 2. Find the table within that div
# 3. Find all 'th' elements where class = sort
headers = soup.find('div', attrs={'class':'chemp margin-top jTable'}).find('table').find_all('th',attrs={'class':'sort'})
 
# Iterate over headers, get the text from each item, and add the results to headers_list
headers_list = []
for header in headers:
    headers_list.append(header.get_text(strip=True))

print(headers_list)

['№', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'xGA', 'xPTS']


In [88]:
body = soup.find('div', attrs={'class':'chemp margin-top jTable'}).table.tbody

#Create a master list for row data
all_rows_list = []
#For each row in the table body
for tr in body.find_all('tr'):
    #Get data from each cell in the row
    row = tr.find_all('td')
    #Create a list to save data from current row
    current_row =[]
    #For each item in the row variable
    for item in row:
        #Add the text data to the current_row list
        current_row.append(item.get_text(strip=True))
    
    #Add the current row data to the master list
    all_rows_list.append(current_row)

#Create dataframe where the rows = all_rows_list and columns = headers_list
xg_df = pd.DataFrame(all_rows_list, columns=headers_list)
#Rename No column to Standings
xg_df = xg_df.rename(columns={"№": "Standing"})
#Add universal delimiter to columns with 'diffs' in them
xg_df['xG'] = xg_df['xG'].str.replace('+','_+')
xg_df['xG'] = xg_df['xG'].str.replace('-','_-')
xg_df['xGA'] = xg_df['xGA'].str.replace('+','_+')
xg_df['xGA'] = xg_df['xGA'].str.replace('-','_-')
xg_df['xPTS'] = xg_df['xPTS'].str.replace('+','_+')
xg_df['xPTS'] = xg_df['xPTS'].str.replace('-','_-')

#Split and create new diff columns based on delimiter
xg_df['xG_diff'] = xg_df['xG'].str.split('_').str.get(1)
xg_df['xGA_diff'] = xg_df['xGA'].str.split('_').str.get(1)
xg_df['xPTS_diff'] = xg_df['xPTS'].str.split('_').str.get(1)

#Cleanup columns by removing diffs
xg_df['xG'] = xg_df['xG'].str.split('_').str.get(0)
xg_df['xGA'] = xg_df['xGA'].str.split('_').str.get(0)
xg_df['xPTS'] = xg_df['xPTS'].str.split('_').str.get(0)

#Convert columns to float or int as possible, ignoring strings
xg_df = xg_df.apply(pd.to_numeric, errors='ignore')
xg_df

,Standing,Team,M,W,D,L,G,GA,PTS,xG,xGA,xPTS,xG_diff,xGA_diff,xPTS_diff
0,1,Liverpool,29,27,1,1,66,21,82,61.03,28.69,59.81,-4.97,7.69,-22.19
1,2,Manchester City,28,18,3,7,68,31,57,73.08,27.87,63.21,5.08,-3.13,6.21
2,3,Leicester,29,16,5,8,58,28,53,47.77,37.43,47.66,-10.23,9.43,-5.34
3,4,Chelsea,29,14,6,9,51,39,48,56.09,31.14,55.55,5.09,-7.86,7.55
4,5,Manchester United,29,12,9,8,44,30,45,49.06,30.56,52.10,5.06,0.56,7.10
5,6,Wolverhampton Wanderers,29,10,13,6,41,34,43,44.62,31.31,49.26,3.62,-2.69,6.26
6,7,Sheffield United,28,11,10,7,30,25,43,36.65,36.88,39.06,6.65,11.88,-3.94
7,8,Tottenham,29,11,8,10,47,40,41,39.59,44.62,37.60,-7.41,4.62,-3.40
8,9,Arsenal,28,9,13,6,40,36,40,38.41,42.43,37.26,-1.59,6.43,-2.74
9,10,Burnley,29,11,6,12,34,40,39,40.37,40.53,39.57,6.37,0.53,0.57


In [93]:
#THIS IS A GENERIC STRUCTURE FOR QUERYING AN API AND LOADING THE JSON

#Define a function to get info from the FPL API and save down
def get_json(file_path):
    r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
    jsonResponse = r.json()
    with open(file_path, 'w') as outfile:
        json.dump(jsonResponse, outfile)
        
#Run the function and choose where to save the json file
get_json('E:/Python_Work/EPL_Data/fpl.json')

#Open the json file and print a list of the keys
with open('E:/Python_Work/EPL_Data/fpl.json') as json_data:
    d = json.load(json_data)
    print(list(d.keys()))

['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types']


In [94]:
df = json_normalize(d['elements'])
print('Columns:\n', list(df), '\n')
print('Dataframe Head:\n', df.head())

Columns:
 ['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'points_per_game', 'second_name', 'selected_by_percent', 'special', 'squad_number', 'status', 'team', 'team_code', 'total_points', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index'] 

Dataframe Head:
    chance_of_playing_next_round  chance_of_playing_this_round    code  \
0                         100.0                         100.0   69140   
1                  

In [95]:
df = json_normalize(d['teams'])
print('Columns:\n', list(df), '\n')
print('Dataframe Head:\n', df.head())

Columns:
 ['code', 'draw', 'form', 'id', 'loss', 'name', 'played', 'points', 'position', 'short_name', 'strength', 'team_division', 'unavailable', 'win', 'strength_overall_home', 'strength_overall_away', 'strength_attack_home', 'strength_attack_away', 'strength_defence_home', 'strength_defence_away'] 

Dataframe Head:
    code  draw  form  id  loss         name  played  points  position  \
0     3     0  None   1     0      Arsenal       0       0         0   
1     7     0  None   2     0  Aston Villa       0       0         0   
2    91     0  None   3     0  Bournemouth       0       0         0   
3    36     0  None   4     0     Brighton       0       0         0   
4    90     0  None   5     0      Burnley       0       0         0   

  short_name  strength team_division  unavailable  win  strength_overall_home  \
0        ARS         4          None        False    0                   1180   
1        AVL         2          None        False    0                   1020   
2  